In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d tongpython/cat-and-dog

In [ ]:
import os
import zipfile

if os.path.exists('/content/cat-and-dog.zip') :
  zip_ref = zipfile.ZipFile('/content/cat-and-dog.zip')
  zip_ref.extractall()
  zip_ref.close()

In [ ]:
import warnings
warnings.simplefilter('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,Flatten,MaxPooling2D,Dense,Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
from kerastuner.engine.hyperparameters import HyperParameters
from keras_tuner import RandomSearch

In [ ]:
img_height = 255
img_width = 255

batch_size = 64
epoch = 10

In [ ]:
train_set =  keras.utils.image_dataset_from_directory('/content/training_set/training_set',
                                                      labels = 'inferred',
                                                      label_mode = 'int',
                                                      seed = 10,
                                                      batch_size = batch_size,
                                                      shuffle = True,
                                                      image_size = (img_height,img_width),
                                                      subset = "training",
                                                      validation_split = 0.2
                                                      )
test_set = keras.utils.image_dataset_from_directory("/content/test_set/test_set",
                                                    labels = 'inferred',
                                                    label_mode = 'int',
                                                    seed = 10,
                                                    batch_size = batch_size,
                                                    shuffle = True,
                                                    image_size = (img_height,img_width),
                                                    subset = "validation",
                                                    validation_split = 0.2
                                        )

In [ ]:
len(train_set)

In [ ]:
train_set

In [ ]:
#normalizing the data
def process(image,label):
  image = tf.cast(image/255.0,tf.float32)
  return image,label

train_set = train_set.map(process)
test_set = test_set.map(process)

In [ ]:
train_iterator = train_set.as_numpy_iterator()

In [ ]:
train_iterator

In [ ]:
batch = train_iterator.next()

In [ ]:
batch[1]

In [ ]:
len(batch)

In [ ]:
batch[0].max()

In [ ]:
#visualizing the data
plt.figure(figsize=(10, 10))
for images, labels in train_set.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
train_set.element_spec

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(filters = hp.Int('conv'+str(i),min_value = 20,max_value = 200),
                       kernel_size = hp.Choice('kernel' + str(i),values = [3,5]),
                       activation = 'relu',
                       input_shape = (255,255,3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Flatten())
    model.add(Dense(units = hp.Int('units',min_value = 20,max_value = 200),
                          activation = 'relu'))
    model.add(Dropout(hp.Choice('dropout',values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer = hp.Choice('optimizers',values = ['adam','rmsprop']),
                 loss = BinaryCrossentropy(),
                 metrics = ['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 3,
    directory = 'classy1',
    project_name = 'DC1'
)

In [ ]:
tuner.search(train_set,epochs = epoch,validation_data = (test_set))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model1 = tuner.get_best_models(num_models = 1)[0]

In [ ]:
model1.fit(train_set,epochs = epoch, initial_epoch = 3,validation_data = test_set)

In [ ]:
import cv2

In [ ]:
test_image = cv2.imread('/content/drive/MyDrive/download.jpg')

In [ ]:
plt.imshow(test_image)

In [ ]:
test_image.shape

In [ ]:
test_image = cv2.resize(test_image,(255,255))

In [ ]:
test_input = test_image.reshape(1,255,255,3)

In [ ]:
model1.predict(test_input)